In [ ]:
%run ./modules/load_data_module.ipynb
%run ./modules/extract_info_clean_data_module.ipynb
%run ./modules/update_dataframe_module.ipynb
%run ./modules/sentiment_module.ipynb
%run ./modules/semantic_similarity_module.ipynb

In [ ]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
import advertools as adv
import plotly.graph_objects as go
import time


pd.options.display.max_colwidth = 285



from textblob import TextBlob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


from spacymoji import Emoji
from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
from sklearn.metrics.pairwise import cosine_similarity



nlp = spacy.load("en_core_web_sm")
emoji = Emoji(nlp)
nlp.add_pipe("emoji", first=True)


stemmer = nltk.SnowballStemmer("english")


# nltk.download('wordnet')
# nltk.download('omw-1.4')


In [ ]:
l= split_dataframe(replies)

In [ ]:
tweets_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*tweets.csv')
users_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*users.csv')
replies_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*replies.csv')
quotes_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*quotes.csv')

In [ ]:
user_dtype = {
    "id": "Int64", 
    "username": "string", 
    "created_at": "string", 
    "name": "string", 
    "protected": "boolean", 
    "verified": "boolean", 
    "followers_count": "Int64", 
    "following_count": "Int64", 
    "tweet_count": "Int64", 
    "listed_count": "Int64", 
    "url": "string", 
    "description": "string"
}


tweet_dtype = {
    "id": "Int64",
    "author_id": "Int64",
    "conversation_id": "Int64",
    "created_at": "string",
    "lang": "string",
    "retweet_count": "Int64",
    "reply_count": "Int64",
    "like_count": "Int64",
    "quote_count": "Int64",
    "reply_settings": "string",
    "text": "string"
}


conversation_dtype = {
    "id": "Int64",
    "author_id": "Int64",
    "conversation_id": "Int64",
    "created_at": "string",
    "lang": "string",
    "retweet_count": "Int64",
    "reply_count": "Int64",
    "like_count": "Int64",
    "quote_count": "Int64",
    "referenced_tweets": "string",
    "text": "string"
}

conversation_dtype_ref_tweet = {
    "id": "Int64",
    "author_id": "Int64",
    "conversation_id": "Int64",
    "created_at": "string",
    "lang": "string",
    "retweet_count": "Int64",
    "reply_count": "Int64",
    "like_count": "Int64",
    "quote_count": "Int64",
    "referenced_tweets": "string",
    "text": "string",
    "reference_tweet_id" : "Int64"
}


In [ ]:
load_tweets = get_df(tweets_path, tweet_dtype)
load_replies = get_df(replies_path, conversation_dtype)
load_quotes = get_df(quotes_path, conversation_dtype).dropna()
load_users = get_df(users_path, user_dtype)


# initial dataframe cleaning

In [ ]:
users = clean_users(load_users)

In [ ]:
load_replies['reference_tweet_id'] = add_reference_id(load_replies)
load_quotes['reference_tweet_id'] = add_reference_id(load_quotes)

### Generate sample data

In [ ]:
# Run this cell to generate new sample data sets

# sample_tweets = get_sample_df(tweets_path, 0.01, tweet_dtype)
# sample_replies = get_sample_reply_quote(sample_tweets, load_replies)
# sample_quotes = get_sample_reply_quote(sample_tweets, load_quotes)

In [ ]:
# print(len(sample_tweets), len(sample_replies), len(sample_quotes)

In [ ]:
# sample_tweets.to_feather('sample_tweets.feather')
# sample_replies.to_feather('sample_replies.feather')
# sample_quotes.to_feather('sample_quotes.feather')

#### load sample data

In [ ]:
sample_tweets = pd.read_feather('/home/niloofar/Developement/Climate-Change/notebooks/sample_tweets.feather')
sample_replies = pd.read_feather('/home/niloofar/Developement/Climate-Change/notebooks/sample_replies.feather')
sample_quotes = pd.read_feather('/home/niloofar/Developement/Climate-Change/notebooks/sample_quotes.feather')

In [ ]:
sample_tweets.dtypes

In [ ]:
print(len(sample_tweets), len(sample_replies), len(sample_quotes))

### Filter the data to load_tweets and first level replies and quotes

In [ ]:
replies = get_sample_reply_quote(load_tweets, load_replies)

In [ ]:
print(len(replies), len(load_quotes))

In [ ]:
replies.to_feather('all_replies.feather')

In [ ]:
replies.dtypes

In [ ]:
# quotes = get_sample_reply_quote(load_tweets, load_quotes)

In [ ]:
# print(len(quotes), len(load_quotes))

In [ ]:
# quotes.dtypes

In [ ]:
# quotes.to_feather('all_quotes_feather.feather')

### load filtered data

In [ ]:
quotes = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/single_dataframes/all_quotes_feather.feather')
quotes.dtypes

In [ ]:
len(quotes)

In [ ]:
replies = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/single_dataframes/all_replies.feather')
replies.dtypes

In [ ]:
len(replies)

In [ ]:
tweets = copy.deepcopy(load_tweets)

In [ ]:
replies = pd.read_feather('/home/niloofar/Developement/single_dataframe/temp_replies.feather')

In [ ]:
quotes = pd.read_feather('/home/niloofar/Developement/single_dataframe/temp_quotes.feather')

# GEN-DATA 

### Add list of hashtags found in tweet/reply/hashtag

In [ ]:
sample_tweets['hashtags'] = add_hashtags(sample_tweets)

In [ ]:
sample_replies['hashtags'] = add_hashtags(sample_replies)

In [ ]:
sample_quotes['hashtags'] = add_hashtags(sample_quotes)

In [ ]:
tweets['hashtags'] = add_hashtags(tweets)

In [ ]:
replies['hashtags'] = add_hashtags(replies)

In [ ]:
quotes['hashtags'] = add_hashtags(quotes)

### Add count of hashtags in tweet/reply/quote

In [ ]:
sample_tweets['hashtag_count'] = add_nbr_hashtags(sample_tweets)

In [ ]:
sample_replies['hashtag_count'] = add_nbr_hashtags(sample_replies)

In [ ]:
sample_quotes['hashtag_count'] = add_nbr_hashtags(sample_quotes)

In [ ]:
tweets['hashtag_count'] = add_nbr_hashtags(tweets)

In [ ]:
replies['hashtag_count'] = add_nbr_hashtags(replies)

In [ ]:
quotes['hashtag_count'] = add_nbr_hashtags(quotes)

### Add number of uppercase characters in tweet/reply/quote

In [ ]:
sample_tweets['uppercase_count'] = add_uppercase_count(sample_tweets)

In [ ]:
sample_replies['uppercase_count'] = add_uppercase_count(sample_replies)

In [ ]:
sample_quotes['uppercase_count'] = add_uppercase_count(sample_quotes)

In [ ]:
tweets['uppercase_count'] = add_uppercase_count(tweets)

In [ ]:
replies['uppercase_count'] = add_uppercase_count(replies)

In [ ]:
quotes['uppercase_count'] = add_uppercase_count(quotes)

### Add percentage of uppercase characters in tweet/reply/quote

In [ ]:
sample_tweets['uppercase_pct'] = add_upper_case_pct(sample_tweets)

In [ ]:
sample_replies['uppercase_pct'] = add_upper_case_pct(sample_replies)

In [ ]:
sample_quotes['uppercase_pct'] = add_upper_case_pct(sample_quotes)

In [ ]:
tweets['uppercase_pct'] = add_upper_case_pct(tweets)

In [ ]:
replies['uppercase_pct'] = add_upper_case_pct(replies)

In [ ]:
quotes['uppercase_pct'] = add_upper_case_pct(quotes)

### Add number of exclamation mark in tweet/reply/quote

In [ ]:
sample_tweets['exclamation_mark_count'] = add_exclamation_mark_count(sample_tweets)

In [ ]:
sample_replies['exclamation_mark_count'] = add_exclamation_mark_count(sample_replies)

In [ ]:
sample_quotes['exclamation_mark_count'] = add_exclamation_mark_count(sample_quotes)

In [ ]:
tweets['exclamation_mark_count'] = add_exclamation_mark_count(tweets)

In [ ]:
replies['exclamation_mark_count'] = add_exclamation_mark_count(replies)

In [ ]:
quotes['exclamation_mark_count'] = add_exclamation_mark_count(quotes)

### Add number of question marks found in tweet/reply/quote

In [ ]:
sample_tweets['question_mark_count'] =  add_question_mark_count(sample_tweets)

In [ ]:
sample_replies['question_mark_count'] =  add_question_mark_count(sample_replies)

In [ ]:
sample_quotes['question_mark_count'] =  add_question_mark_count(sample_quotes)

In [ ]:
tweets['question_mark_count'] =  add_question_mark_count(tweets)

In [ ]:
replies['question_mark_count'] =  add_question_mark_count(replies)

In [ ]:
quotes['question_mark_count'] =  add_question_mark_count(quotes)

### Add number of URLs found in tweet/reply/quote

In [ ]:
sample_tweets['url_count'] = add_url_count(sample_tweets)

In [ ]:
sample_replies['url_count'] = add_url_count(sample_replies)

In [ ]:
sample_quotes['url_count'] = add_url_count(sample_quotes)

In [ ]:
tweets['url_count'] = add_url_count(tweets)

In [ ]:
replies['url_count'] = add_url_count(replies)

In [ ]:
quotes['url_count'] = add_url_count(quotes)

### Add number of mentions in tweet/reply/quote

In [ ]:
sample_tweets['mention_count'] = add_mention_count(sample_tweets)

In [ ]:
sample_replies['mention_count'] = add_mention_count(sample_replies)

In [ ]:
sample_quotes['mention_count'] = add_mention_count(sample_quotes)

In [ ]:
tweets['mention_count'] = add_mention_count(tweets)

In [ ]:
replies['mention_count'] = add_mention_count(replies)

In [ ]:
quotes['mention_count'] = add_mention_count(quotes)

### Add number of emojis found in tweet/reply/quote

In [ ]:
sample_tweets['emojie_count'] = add_emoji_count(sample_tweets)

In [ ]:
sample_replies['emojie_count'] = add_emoji_count(sample_replies)

In [ ]:
sample_quotes['emojie_count'] = add_emoji_count(sample_quotes)

In [ ]:
tweets['emojie_count'] = add_emoji_count(tweets)

In [ ]:
replies['emojie_count'] = add_emoji_count(replies)

In [ ]:
quotes['emojie_count'] = add_emoji_count(quotes)

### Add tweet's author number of followers

In [ ]:
sample_tweets['followers_count'] = add_followers_count(sample_tweets, users)

In [ ]:
tweets['followers_count'] = add_followers_count(tweets, users)

### Add engagement score

In [ ]:
sample_tweets['engagement_score'] = add_engagement_score(sample_tweets)

In [ ]:
tweets['engagement_score'] = add_engagement_score(tweets)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
sample_tweets.head(3)

In [ ]:
sample_replies.head(3)

In [ ]:
sample_quotes.head(3)

## sentiment

### Add sentiment score to tweet/reply/quote

In [ ]:
sample_tweets['sentiment_score'] = add_sentiment_score(sample_tweets)

In [ ]:
sample_replies['sentiment_score'] = add_sentiment_score(sample_replies)

In [ ]:
sample_quotes['sentiment_score'] = add_sentiment_score(sample_quotes)

In [ ]:
tweets['sentiment_score'] = add_sentiment_score(tweets)

In [ ]:
replies['sentiment_score'] = add_sentiment_score(replies)

In [ ]:
quotes['sentiment_score'] = add_sentiment_score(quotes)

### Add sentiment cateory (pos, neg, neu) to tweet/reply/quote

In [ ]:
sample_tweets['sentiment_category'] = add_sentiment_category(sample_tweets)

In [ ]:
sample_replies['sentiment_category'] = add_sentiment_category(sample_replies)

In [ ]:
sample_quotes['sentiment_category'] = add_sentiment_category(sample_quotes)

In [ ]:
tweets['sentiment_category'] = add_sentiment_category(tweets)

In [ ]:
replies['sentiment_category'] = add_sentiment_category(replies)

In [ ]:
quotes['sentiment_category'] = add_sentiment_category(quotes)

### Add average sentiment of replies for each tweet

In [ ]:
sample_tweets['avg_replies_sentiment'] = add_average_conversation_sentiment(sample_tweets, sample_replies)

In [ ]:
sample_tweets['avg_quotes_sentiment'] = add_average_conversation_sentiment(sample_tweets, sample_quotes)

In [ ]:
tweets['avg_replies_sentiment'] = add_average_conversation_sentiment(tweets, replies)

In [ ]:
tweets['avg_quotes_sentiment'] = add_average_conversation_sentiment(tweets, quotes)

### Add average sentiment of replies + quotes of each tweet

In [ ]:
sample_tweets['avg_conversation_sentiment'] = add_average_conversation_sentiment(sample_tweets, pd.concat([sample_replies, sample_quotes]))

In [ ]:
tweets['avg_conversation_sentiment'] = add_average_conversation_sentiment(tweets, pd.concat([replies, quotes]))

### Add variance of replies/quotes/replies+quotes sentiment

In [ ]:
sample_tweets['var_replies_sentiment'] = add_variance_of_conversation_sentiment(sample_tweets, sample_replies)

In [ ]:
sample_tweets['var_quotes_sentiment'] = add_variance_of_conversation_sentiment(sample_tweets, sample_quotes)

In [ ]:
sample_tweets['var_conversation_sentiment'] = add_variance_of_conversation_sentiment(sample_tweets, pd.concat([sample_replies, sample_quotes]))

In [ ]:
tweets['var_replies_sentiment'] = add_variance_of_conversation_sentiment(tweets, replies)

In [ ]:
tweets['var_quotes_sentiment'] = add_variance_of_conversation_sentiment(tweets, quotes)

In [ ]:
tweets['var_conversation_sentiment'] = add_variance_of_conversation_sentiment(tweets, pd.concat([replies, quotes]))

### Add percentage of replies/quotes with similar sentiment to the reference tweet

In [ ]:
sample_tweets['pct_similar_sentiment_replies'] = add_pct_sentiment_category(sample_tweets, sample_replies)

In [ ]:
sample_tweets['pct_similar_sentiment_quotes'] = add_pct_sentiment_category(sample_tweets, sample_quotes)

In [ ]:
tweets['pct_similar_sentiment_replies'] = add_pct_sentiment_category(tweets, replies)

In [ ]:
tweets['pct_similar_sentiment_quotes'] = add_pct_sentiment_category(tweets, quotes)

In [ ]:
sample_tweets.head(3)

In [ ]:
sample_replies.head(3)

In [ ]:
sample_quotes.head(3)

### text cleaning

In [ ]:
sample_tweets['clean_text'] = add_clean_text(sample_tweets)

In [ ]:
sample_replies['clean_text'] = add_clean_text(sample_replies)

In [ ]:
sample_quotes['clean_text'] =  add_clean_text(sample_quotes)

In [ ]:
tweets['clean_text'] = add_clean_text(tweets)

In [ ]:
replies['clean_text'] = add_clean_text(replies)

In [ ]:
quotes['clean_text'] =  add_clean_text(quotes)

### Add tweet to conversation Notice, add clean text to conversation dataframe first!

In [ ]:
sample_replies['clean_tweet'] = add_tweet_to_conversation(sample_tweets, sample_replies)

In [ ]:
sample_quotes['clean_tweet'] = add_tweet_to_conversation(sample_tweets, sample_quotes)

In [ ]:
replies['clean_tweet'] = add_tweet_to_conversation(tweets, replies)

In [ ]:
quotes['clean_tweet'] = add_tweet_to_conversation(tweets, quotes)

In [ ]:
replies.to_feather('temp_replies.feather')

In [ ]:
quotes.to_feather('temp_quotes.feather')

In [ ]:
tweets.to_feather('temp_tweets.feather')

### Add sentiment similarity to conversations

In [ ]:
sample_replies['semantic_similarity'] = add_tweet_conv_semantic_similarity(sample_replies['clean_tweet'], sample_replies['clean_text'])

In [ ]:
sample_quotes['semantic_similarity'] = add_tweet_conv_semantic_similarity(sample_quotes['clean_tweet'], sample_quotes['clean_text'])

In [ ]:
# Do not use this anymore!!!
# replies['semantic_similarity'] = add_tweet_conv_semantic_similarity(replies['clean_tweet'], replies['clean_text'])

In [ ]:
# Do not use this anymore!!!
# quotes['semantic_similarity'] = add_tweet_conv_semantic_similarity(quotes['clean_tweet'], quotes['clean_text'])

In [ ]:
replies.to_feather('replies_final_version.feather')

In [ ]:
quotes.to_feather('quotes_final_version.feather')

### Add variance of semantic similarity scores 

In [ ]:
sample_tweets['var_replies_semantic'] = add_variance_of_conversation_sentiment(sample_tweets, sample_replies)

In [ ]:
sample_tweets['var_quotes_semantic'] = add_variance_of_conversation_sentiment(sample_tweets, sample_quotes)

In [ ]:
sample_tweets['var_conversation_semantic'] = add_variance_of_conversation_sentiment(sample_tweets, pd.concat([sample_quotes, sample_replies]))

In [ ]:
tweets['var_replies_semantic'] = add_variance_of_conversation_sentiment(tweets, replies)

In [ ]:
tweets['var_quotes_semantic'] = add_variance_of_conversation_sentiment(tweets, quotes)

In [ ]:
tweets['var_conversation_semantic'] = add_variance_of_conversation_sentiment(tweets, pd.concat([quotes, replies]))

In [ ]:
tweets.to_feather('tweets_final_version.feather')

In [ ]:
check_sample_quo = pd.read_feather('/home/niloofar/Developement/Climate-Change/notebooks/sample_quotes_final_version.feather')